In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver

# Manthan's Data

In [6]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
base_url = 'https://www.espncricinfo.com/series/ipl-2016-968923/match-schedule-fixtures-and-results'
s16 = requests.get(base_url,headers=headers)
print(s16.status_code)
soup = BeautifulSoup(s16.text,'html.parser')

200


In [7]:
s16m = []
anchors = soup.find_all('a',class_='ds-inline-flex ds-items-start ds-leading-none')
for anchor in anchors :
    link = anchor.get('href')
    link = urljoin(base_url,link)
    if 'live-cricket-score' in link :
        s16m.append(link)
s16m

['https://www.espncricinfo.com/series/ipl-2016-968923/mumbai-indians-vs-rising-pune-supergiants-1st-match-980901/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-delhi-daredevils-2nd-match-980903/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/kings-xi-punjab-vs-gujarat-lions-3rd-match-980905/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-vs-sunrisers-hyderabad-4th-match-980907/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/kolkata-knight-riders-vs-mumbai-indians-5th-match-980909/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/gujarat-lions-vs-rising-pune-supergiants-6th-match-980911/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/delhi-daredevils-vs-kings-xi-punjab-7th-match-980913/live-cricket-score',
 'https://www.espncricinfo.com/series/ipl-2016-968923/sunrisers-hyderab

Columns to fill
1. Match type
2. POTM
3. Venue
4. Team1
5. Team2
6. Toss Winner
7. Toss Decision
8. Winner
9. Win margin
10. Result type(number of wickets left or run difference)
11. Target runs

In [9]:
match_data = {
        'Match Type': '',
        'POTM': '',
        'Venue': '',
        'Team1': '',
        'Team2': '',
        'Toss_Winner': '',
        'Toss_Decision': '',
        'Winner': '',
        'Win_Margin': '',
        'Result_Type': '',
        'Target_Runs': ''
}

## Match Type, Venue and POTM

In [11]:
match = requests.get(s16m[0],headers=headers)
print(match.status_code)
soup_match = BeautifulSoup(match.text,'html.parser')

200


In [12]:
driver = webdriver.Chrome()
driver.get(s16m[0])

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

elements = soup.find_all('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')[0].get_text(strip=True)
potmscrape = soup.find_all('span', class_='ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer')[0].get_text(strip=True)
teamscrape = soup.find_all('span',class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
tossscrape = soup.find_all('td',class_='ds-text-typo')[0].find('span',class_='ds-text-tight-s ds-font-regular').get_text(strip=True)

driver.quit()

In [13]:
venue = elements.split(',')[1].strip()
match_data['Venue'] = venue

matching_word = next((word for word in ['Final','Qualifier','Eliminator'] if word in elements), None)

if matching_word:
    match_data['Match Type'] = matching_word
else:
    match_data['Match Type'] = 'League'
match_data['POTM'] = potmscrape

## Teams

In [15]:
match_data['Team1'] = teamscrape[0].get_text(strip=True)
match_data['Team2'] = teamscrape[1].get_text(strip=True)

{'Match Type': 'League',
 'POTM': 'Ajinkya Rahane',
 'Venue': 'Wankhede',
 'Team1': 'MI',
 'Team2': 'RPS',
 'Toss_Winner': '',
 'Toss_Decision': '',
 'Winner': '',
 'Win_Margin': '',
 'Result_Type': '',
 'Target_Runs': ''}

## Toss Winner and Decision

In [32]:
match_data['Toss_Winner'] = toss.split(',')[0]
if 'bat' in toss :    
    match_data['Toss_Decision'] = 'bat'
else :
    match_data['Toss_Decision'] = 'field'
match_data

{'Match Type': 'League',
 'POTM': 'Ajinkya Rahane',
 'Venue': 'Wankhede',
 'Team1': 'MI',
 'Team2': 'RPS',
 'Toss_Winner': 'Mumbai Indians',
 'Toss_Decision': 'bat',
 'Winner': '',
 'Win_Margin': '',
 'Result_Type': '',
 'Target_Runs': ''}